In [1]:
import requests, json
import pandas as pd

In [2]:
def list_of_dicts_to_nested_dict(data):
    nested_dict = {}
    for item in data:
        current_dict = nested_dict
        for key, value in item.items():
            if key == 'agregados':
                if key not in current_dict:
                    current_dict[key] = []
                current_dict[key].extend(value)
            else:
                if key not in current_dict:
                    current_dict[key] = {}  # Create an empty nested dictionary if key doesn't exist
                current_dict = current_dict[key]
        if 'agregados' in current_dict:
            current_dict['agregados'] = list_of_dicts_to_nested_dict(current_dict['agregados'])
    return nested_dict

In [45]:
# Código do municipio Embu-Guaçu na base IBGE
cod_ibge = '3515103'

# Dados demográficos - IBGE

## População de Embu-Guaçu (2010-2022)

In [18]:
url_base = 'https://servicodados.ibge.gov.br/api/v3/agregados'

## Listar dados disponíveis para município

In [4]:
# identificador nivel municipal
nivel = 'N6'

response = requests.get(f'{url_base}?nivel={nivel}')

data_json = response.json()

for conjunto_dataset in data_json:
    print(conjunto_dataset['nome'], conjunto_dataset['id'])

Áreas Urbanizadas D5
Cadastro Central de Empresas CL
Censo Agropecuário CA
Censo Demográfico CD
Contagem da População  CM
Estatísticas dos Cadastros de Microempreendedores Individuais D9
Estimativas de População XF
Fundações Privadas e Associações Sem Fins Lucrativos FP
Indicadores de Desenvolvimento Sustentável IU
Índice de Preços ao Consumidor em Real IL
Índice de Reajuste do Salário Mínimo IJ
Índice Nacional de Preços ao Consumidor PC
Índice Nacional de Preços ao Consumidor Amplo IA
Índice Nacional de Preços ao Consumidor Amplo 15 IQ
Índice Nacional de Preços ao Consumidor Amplo Especial IG
Índice Nacional de Preços ao Consumidor Especial PG
Objetivos de Desenvolvimento Sustentável C2
Pesquisa da Pecuária Municipal PP
Pesquisa de Estoques ES
Pesquisa de Informações Básicas Municipais PM
Pesquisa de Orçamentos Familiares OF
Pesquisa de Serviços de Hospedagem SH
Pesquisa Estatísticas do Registro Civil RC
Pesquisa Nacional de Saneamento Básico SB
Pesquisa Nacional de Saúde XN
Pesquisa 

## Filtro de dasets relevantes

In [5]:
datasets_selecionados = ['CD','CM','XF','PM','OF','SB','XN','AA']

In [6]:
conj_datasets_selecionados = [dataset for dataset in data_json if dataset['id'] in datasets_selecionados]

In [7]:
conj_datasets_selecionados[0]

{'id': 'CD',
 'nome': 'Censo Demográfico',
 'agregados': [{'id': '102',
   'nome': 'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal, total e que tiveram filhos, e os filhos tidos pelas mulheres de 10 anos ou mais de idade, que viviam em união conjugal, os nascidos vivos, os nascidos vivos no período de referência de 12 meses, nascidos mortos e os que estavam vivos na data de referência, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres'},
  {'id': '103',
   'nome': 'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal e tiveram filhos nascidos vivos, por número de filhos tidos nascidos vivos, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres'},
  {'id': '105',
   'nome': 'Filhos tidos nascidos vivos das mulheres de 10 anos ou mais de idade, que viviam em união conjugal, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulhere

## Listar anos disponíveis para cada conjunto de datasets

In [8]:
dict_anos_datasets = {}

for dataset in datasets_selecionados:
    list_anos = []

    response = requests.get(f'{url_base}?acervo=/S/{dataset}/P/Q')
    data_json = response.json()

    for ano in data_json:
        list_anos.append(ano['id'])

    dict_anos_datasets[dataset] = list_anos

In [9]:
dict_anos_datasets

{'CD': ['1872',
  '1890',
  '1900',
  '1920',
  '1940',
  '1950',
  '1960',
  '1970',
  '1980',
  '1991',
  '2000',
  '2010',
  '2022'],
 'CM': ['1996', '2007'],
 'XF': ['2001',
  '2002',
  '2003',
  '2004',
  '2005',
  '2006',
  '2008',
  '2009',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018',
  '2019',
  '2020',
  '2021'],
 'PM': ['2005', '2020', '2021'],
 'OF': ['1987', '1995', '2002', '2008', '2018'],
 'SB': ['2000', '2008', '2017'],
 'XN': ['2013', '2019'],
 'AA': ['2009', '2012', '2015']}

In [21]:
# Neste caso, vamos trabalhar apenas com datasets que tiveram atualização recente
anos_datasets_selecionados = {'CD':'2022','XF':'2021','PM':'2021','XN':'2019'}

## Listar tabelas disponíveis para os anos e datasets selecionados

In [28]:
dicionario_tabelas_datasets = {}

for dataset in anos_datasets_selecionados.items():
    if isinstance(dataset[1], list):
        print(f'{url_base}?acervo=/S/{dataset[0]}/P/{dataset[1][0]}/P/{dataset[1][1]}/T/Q?_1700211710044')
        response = requests.get(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1][0]}/P/1_{dataset[1][1]}/T/Q?_=1700211710044')
        data_json = response.json()
        
        dicionario_tabelas_datasets[dataset[0]] = data_json
    else:
        periodo = f'1_{dataset[1]}'
        print(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1]}/T/Q?_=1700211710044')
        response = requests.get(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1]}/T/Q?_=1700211710044')
        data_json = response.json()
        dicionario_tabelas_datasets[dataset[0]] = data_json

https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/CD/P/1_2022/T/Q?_=1700211710044
https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/XF/P/1_2021/T/Q?_=1700211710044
https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/PM/P/1_2021/T/Q?_=1700211710044
https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/XN/P/1_2019/T/Q?_=1700211710044


Após exploração dos datasets no site do [IBGE](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3), foi constatado que apenas o dataset do Censo Demográfico (CD) realmente possui dados atuais e representativos do contexto específico de Embu-Guaçu

# Requisições dados - Censo Demográfico IBGE

In [42]:
# Dicionário com tabelas disponíveis para requisição
dicionario_tabelas_datasets['CD'][:4]

[{'id': '1209', 'literal': 'População, por grupos de idade'},
 {'id': '4709',
  'literal': 'População residente, Variação absoluta de população residente e Taxa de crescimento geométrico'},
 {'id': '4711', 'literal': 'Domicílios recenseados, por espécie'},
 {'id': '4712',
  'literal': 'Domicílios particulares permanentes ocupados, Moradores em domicílios particulares permanentes ocupados e Média de moradores em domicílios particulares permanentes ocupados'}]

In [43]:
df_tabelas_censo = pd.json_normalize(dicionario_tabelas_datasets['CD'])

Lista de tabelas consultadas

- 9514- População residente, por sexo, idade e forma de declaração da idade
- 9520- População residente, Variação absoluta da população residente e Taxa de crescimento geométrico (segunda apuração)
- 9606- População residente, por cor ou raça, segundo o sexo e a idade
- 9521- Domicílios particulares permanentes ocupados, Moradores em domicílios particulares permanentes ocupados e Média de moradores em domicílios particulares permanentes ocupados (segunda apuração)
- 9397- Moradores em domicílios particulares permanentes ocupados, por existência de banheiro ou sanitário e número de banheiros de uso exclusivo do domicílio e por tipo de esgotamento sanitário, segundo grupos de idade e cor ou raça
- 9522- População residente, Área territorial e Densidade demográfica (segunda apuração)

In [46]:
# Lista de códigos das tabelas que serão consultadas na API
list_tabelas_censo = [str(x) for x in [9514, 9520, 9606, 9521, 9397, 9522]]
list_tabelas_censo

['9514', '9520', '9606', '9521', '9397', '9522']

## Lista de metadados para cada tabela

In [47]:
url_base

'https://servicodados.ibge.gov.br/api/v3/agregados'

In [76]:
dicionario_variaveis_tabelas = {}

for tabela in list_tabelas_censo:
    response = requests.get(f'{url_base}/{tabela}/metadados')
    data_json = response.json()['variaveis']

    print(data_json)

    dicionario_variaveis_tabelas[tabela] = {}

    for item in data_json:
        dicionario_variaveis_tabelas[tabela][str(item['id'])] = item['nome']

[{'id': 93, 'nome': 'População residente', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 1000093, 'nome': 'População residente - percentual do total geral', 'unidade': '%', 'sumarizacao': []}]
[{'id': 93, 'nome': 'População residente', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 5936, 'nome': 'Variação absoluta da população residente 2010 compatibilizada', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 10605, 'nome': 'Taxa de crescimento geométrico', 'unidade': '%', 'sumarizacao': []}]
[{'id': 93, 'nome': 'População residente', 'unidade': 'Pessoas', 'sumarizacao': ['nivelTerritorial']}, {'id': 1000093, 'nome': 'População residente - percentual do total geral', 'unidade': '%', 'sumarizacao': []}]
[{'id': 381, 'nome': 'Domicílios particulares permanentes ocupados', 'unidade': 'Domicílios', 'sumarizacao': []}, {'id': 382, 'nome': 'Moradores em domicílios particulares permanentes ocupados', 'unidade': 'Pessoas', 'sumarizacao': []}, {'id': 5930, 'nome': 'Média de moradores em domi

In [82]:
dicionario_variaveis_tabelas

{'9514': {'93': 'População residente',
  '1000093': 'População residente - percentual do total geral'},
 '9520': {'93': 'População residente',
  '5936': 'Variação absoluta da população residente 2010 compatibilizada',
  '10605': 'Taxa de crescimento geométrico'},
 '9606': {'93': 'População residente',
  '1000093': 'População residente - percentual do total geral'},
 '9521': {'381': 'Domicílios particulares permanentes ocupados',
  '382': 'Moradores em domicílios particulares permanentes ocupados',
  '5930': 'Média de moradores em domicílios particulares permanentes ocupados'},
 '9397': {'382': 'Moradores em domicílios particulares permanentes ocupados',
  '1000382': 'Moradores em domicílios particulares permanentes ocupados - percentual do total geral'},
 '9522': {'93': 'População residente',
  '6318': 'Área da unidade territorial',
  '614': 'Densidade demográfica'}}

In [71]:
pd.json_normalize(dicionario_variaveis_tabelas)

,9514.93,9514.1000093,9520.93,9520.5936,9520.10605,9606.93,9606.1000093,9521.381,9521.382,9521.5930,9397.382,9397.1000382,9522.93,9522.6318,9522.614
0,População residente,População residente - percentual do total geral,População residente,Variação absoluta da população residente 2010 ...,Taxa de crescimento geométrico,População residente,População residente - percentual do total geral,Domicílios particulares permanentes ocupados,Moradores em domicílios particulares permanent...,Média de moradores em domicílios particulares ...,Moradores em domicílios particulares permanent...,Moradores em domicílios particulares permanent...,População residente,Área da unidade territorial,Densidade demográfica


# 9514: População residente, por sexo, idade e forma de declaração da idade

In [86]:
tabela = list_tabelas_censo[0]
variaveis = list(dicionario_variaveis_tabelas[tabela].keys())
str_variaveis = '|'.join(variaveis)

print(tabela)
print(str_variaveis)

9514
93|1000093


In [92]:
url_final = f'{url_base}/{tabela}/periodos/2022/variaveis/{str_variaveis}?localidades=N6[{cod_ibge}]&classificacao=2[all]|287[all]'
print(url_final)
response = requests.get(url_final)
data_json = response.json()

print(data_json)

https://servicodados.ibge.gov.br/api/v3/agregados/9514/periodos/2022/variaveis/93|1000093?localidades=N6[3515103]&classificacao=2[all]|287[all]
[{'id': '93', 'variavel': 'População residente', 'unidade': 'Pessoas', 'resultados': [{'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'100362': 'Total'}}, {'id': '286', 'nome': 'Forma de declaração da idade', 'categoria': {'113635': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '66970'}}]}, {'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'93070': '0 a 4 anos'}}, {'id': '286', 'nome': 'Forma de declaração da idade', 'categoria': {'113635': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '4079'}}]},

In [131]:
for result in data_json[0]['resultados']:
    print(result)

    for classif in result['classificacoes']:
        var_name = classif['nome']
        var_class = list(classif['categoria'].values())[0]
        var_value = float(list(result['series'][-1]['serie'].values())[0])

        print(var_name)
        print(var_class)
        print(var_value)
        print('\n')

    print('----------------')

{'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'100362': 'Total'}}, {'id': '286', 'nome': 'Forma de declaração da idade', 'categoria': {'113635': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '66970'}}]}
Sexo
Total
66970.0


Idade
Total
66970.0


Forma de declaração da idade
Total
66970.0


----------------
{'classificacoes': [{'id': '2', 'nome': 'Sexo', 'categoria': {'6794': 'Total'}}, {'id': '287', 'nome': 'Idade', 'categoria': {'93070': '0 a 4 anos'}}, {'id': '286', 'nome': 'Forma de declaração da idade', 'categoria': {'113635': 'Total'}}], 'series': [{'localidade': {'id': '3515103', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Embu-Guaçu (SP)'}, 'serie': {'2022': '4079'}}]}
Sexo
Total
4079.0


Idade
0 a 4 anos
4079.0


Forma de declaração da idade
Total
4079.0


----------------
{'classificacoes': 

In [114]:
data_json[0]['resultados'][0]['series']

[{'localidade': {'id': '3515103',
   'nivel': {'id': 'N6', 'nome': 'Município'},
   'nome': 'Embu-Guaçu (SP)'},
  'serie': {'2022': '66970'}}]